In [4]:
import os, json, numpy as np
import pandas as pd
import sys, re   
import glob
sys.path.insert(1, 'D:\DystoniaCoalition\DystViz')   
from WebApp.utils import MP4toJson

import numpy as np
import cv2, pickle
cap = None

import matplotlib.pyplot as plt

ret, frame , pid, score =None, None, None, None # Read the frame

openposeOutputDirectory = "C:\DystoniaCoalition\OpenPoseFullFrame\\"

neckData = pd.read_csv("D:\DystoniaCoalition\processed\ExportNeckCSV.csv",)


GDRS = pd.read_excel("D:\DystoniaCoalition\dataset\P1.LV.Koirala.2021.3.30-2_edited.xlsx", sheet_name='GDRS')
GDRS.columns
GDRS.drop(GDRS.columns[[1,2,3,4]],axis=1,inplace=True)
GDRS.rename(columns={ GDRS.columns[0]: "PID" }, inplace = True)
GDRS = GDRS.loc[:, ~GDRS.columns.str.startswith('Unnamed')].iloc[1:].reset_index(drop=True)
GDRS['PID'] = GDRS['PID'].apply(lambda x: re.findall("\d+", x)[0])
# GDRS

def plotScore():
    global score
    score = GDRS[GDRS['PID']==str(pid)].values[0][4:6]
    return plt.text(120, 300, 
             'PID: '+ str(pid)+'\nLarynx: '+ str(score[0])+ '\nNeck: '+ str(score[1]),
             horizontalalignment='center', bbox=dict(facecolor='white', alpha=0.5))

### Processings for frame  normalization to 120

In [5]:
# _openposeOutputsMain =  os.listdir(openposeOutputDirectory) 

# total = {}
# for openposeOutputsMain in _openposeOutputsMain:
#     global pid, score
#     ## choose one of the file for now
    
#     videoFilePath = os.path.join("D:\DystoniaCoalition\RawVideos\\", openposeOutputsMain)
#     # print(openposeOutputsMain)
#     print("", end='.')


#     #get pid
#     try:
#         pid = int(openposeOutputsMain.split("_")[0].replace("DYS", " "))
#     except:
#         pid = int(openposeOutputsMain.split(" ")[0].replace("DYS", " "))

#     TotalFrames, JsonDict = MP4toJson(openposeOutputsMain, openposeOutputDirectory)
#     _JsonDict = dict(JsonDict)  #same thing in dict format
#     # plotallpoints()
#     # exportGIF()
#     total[pid]=JsonDict

# # DUMP AND GET DATA
# import pickle
# with open('D:\DystoniaCoalition\processed\\annotatios_RawCroppedFramesAll.pickle', 'wb') as handle:
#     pickle.dump(total, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('D:\DystoniaCoalition\processed\\annotatios_RawCroppedFramesAll.pickle', 'rb') as handle:
#     _data = pickle.load(handle)


# # CONVERT DATA TO PANDAS DF
# from itertools import islice
# import os
# from IPython.display import clear_output


# allScores = []
# from operator import itemgetter
# colu= [("Dystonia",'PID'), ("Dystonia",'FID'), ("NOSE",'x'),("NOSE",'y'), ("NECK",'x'),("NECK",'y'), ("RSHO",'x'),("RSHO",'y'), 
#        ("RELB",'x'),("RELB",'y'), ("RWRI",'x'),("RWRI",'y'), ("LSHO",'x'),("LSHO",'y'), ("LELB",'x'),("LELB",'y'), 
#        ("LWRI",'x'),("LWRI",'y'), ("MHIP",'x'),("MHIP",'y'), ("RHIP",'x'),("RHIP",'y'), ("RKNE",'x'),("RKNE",'y'), ("RANK",'x'),
#        ("RANK",'y'), ("LHIP",'x'),("LHIP",'y'), ("LKNE",'x'),("LKNE",'y'), ("LANK",'x'),("LANK",'y'), ("REYE",'x'),
#        ("REYE",'y'), ("LEYE",'x'),("LEYE",'y'), ("REAR",'x'),("REAR",'y'), ("LEAR",'x'),("LEAR",'y'), ("LBTO",'x'),("LBTO",'y'), 
#        ("LSTO",'x'),("LSTO",'y'), ("LHEL",'x'),("LHEL",'y'), ("RBTO",'x'),("RBTO",'y'), ("RSTO",'x'),("RSTO",'y'), 
#        ("RHEL",'x'),("RHEL",'y'),]

# rows_list  = []
# for key, value in _data.items():
#     for framex in value:
#         if len(framex):
#             for coords in framex[1]:
#                 if (coords):
#                     dict1 ={ ("Dystonia",'PID'): key,  ("Dystonia",'FID'): framex[0],}
#                     for coord in coords:
#                         dict1[colu[int(coord[2])*2+2]]=coord[0]
#                         dict1[colu[int(coord[2])*2+3]]= coord[1]
#                     rows_list.append(dict1)
# df_fullframe = pd.DataFrame(rows_list)
# df_fullframe.columns =  pd.MultiIndex.from_tuples(df_fullframe.columns, names=['Part', 'Coord'])
# df_fullframe.to_pickle("./df_fullframe.pkl")
  
# df_fullframe = pd.read_pickle("./df_fullframe.pkl") 

# #filter to 120 frames
# sample_num = 120
# df_fixedframe = df_fullframe.loc[df_fullframe.index.intersection([x for b in 
#                 [[_df.index[int(i)] for i in  np.linspace(0, len(_df.index)-1, sample_num)] 
#                  for name, _df in df_fullframe.dropna(subset=[('NECK', 'x')]).dropna(subset=[('NOSE', 'x')])
#                                                 .groupby(('Dystonia','PID'))]
#             for x in b])]

# df_fixedframe.to_pickle("./df_fixedframe.pkl")  

In [6]:
df_fixedframe = pd.read_pickle("./df_fixedframe.pkl")

## Plot all point at once

In [7]:
import matplotlib.pyplot as plt

def plotallpoints():
    global cap, ret, frame, score, JsonDict, videoFilePath
    cap = cv2.VideoCapture(videoFilePath) #video_name is the video being called
    cap.set(1,JsonDict[0][0]); # Where frame_no is the frame you want
    ret, frame = cap.read() # Read the frame

    plt.figure(figsize=(10,10))
    #check if has Keypoints
    # kp = _JsonDict.get(int(cap.get(cv2.CAP_PROP_POS_FRAMES)))[0]
    # if _JsonDict.get(int(cap.get(cv2.CAP_PROP_POS_FRAMES)))[0]:
    data = {'x': [e[0] for idx, eachFrame in enumerate(JsonDict) for p in eachFrame[1] for e in p],
            'y': [e[1] for idx, eachFrame in enumerate(JsonDict) for p in eachFrame[1] for e in p],}
    plt.scatter(x=data["x"], y=data["y"], color="red", s=0.1)
    plt.rcParams["figure.figsize"] = (20,3)
    plotScore()
    plt.imshow(frame)
    # plt.show()
    plt.savefig("D:\DystoniaCoalition\processed\gifs\\total\\DYS_"+str(pid)+"_Neck_"+ str(score[1])+ ".png" )
    plt.clf()
# plotallpoints()

In [8]:
def exportGIF():
    global cap, ret, frame, score, JsonDict, videoFilePath

    ims=[]
    import matplotlib.animation as animation

    fig = plt.figure()

    cap.set(1,JsonDict[0][0]); # Where frame_no is the frame you want

    for frm in JsonDict:
        kp = frm[0]
        cap.set(1,kp)
        ret, frame = cap.read()
        ims.append([plt.imshow(frame), plotScore(),
                    plt.scatter(x=[e[0] for e in frm[1][0]], y=[e[1] for e in frm[1][0]], color="red", s=10) ])

    ani = animation.ArtistAnimation(fig, ims, interval=200, blit=True, 
                                    repeat_delay=500)
    ani.save("D:\DystoniaCoalition\processed\gifs\\DYS_"+str(pid)+"_Neck_"+ str(score[1])+ ".gif", writer='imagemagick', dpi=60)
    plt.clf()
# exportGIF()

In [9]:
# # clasic method for  plotting: loop ovr videos
# _openposeOutputsMain =  os.listdir(openposeOutputDirectory) 
# for openposeOutputsMain in _openposeOutputsMain:
#     global pid, score
#     videoFilePath = os.path.join("D:\DystoniaCoalition\RawVideos\\", openposeOutputsMain)
#     videoFilePath = "D:\DystoniaCoalition\RawVideos\DYS1001_20170222.mp4"
#     # print(openposeOutputsMain)
#     print("", end='.')
#     #get pid
#     try:
#         pid = int(openposeOutputsMain.split("_")[0].replace("DYS", " "))
#     except:
#         pid = int(openposeOutputsMain.split(" ")[0].replace("DYS", " "))

#     JsonDict = ff_data[pid]
#     _JsonDict = dict(JsonDict)  #same thing in dict format
#     plotallpoints()
#     exportGIF()

# Loop to create animations

In [10]:
toFixedFrame = False
if toFixedFrame:
    for name, _df in df_fixedframe.groupby(('Dystonia','PID')):
        # global pid, score, videoFilePath, 
        _videoFilePath = glob.glob("D:\DystoniaCoalition\RawVideos\\DYS"+ str(name)+"_*.mp4")
        assert len(_videoFilePath)==1
        videoFilePath = _videoFilePath[0]  
        pid = int(name)
        # print("", end='.')
        TOJsonDict = []
        for r in _df.iterrows():
            coordinates =[]
            # print("sur", list(r[1]), "..")
            for idx, each in enumerate(zip(list(r[1])[::2],list(r[1])[1::2])):
                # print(idx, each, idx>=1 and isinstance(each, tuple))
                if (idx>=1 and isinstance(each, tuple)):
                    coordinates.append([each[0], each[1], int(idx)-1])
            
            TOJsonDict.append([r[1][1],[coordinates] ] )
            # print(coordinates)
            
        JsonDict = TOJsonDict
        _JsonDict = dict(JsonDict) 
        plotallpoints()
        exportGIF()

# Forward Fill the missing value

In [11]:
df_fixed8 = df_fixedframe.loc[:,  df_fixedframe.columns.get_level_values(0).isin(['Dystonia', 'NOSE', 'RSHO', 'LSHO','NECK',  'REYE', 'LEYE', 'REAR', 'LEAR'])]
# print("Missing Values in each coorinates:")
# print(df_fixed8.isna().sum()[::2])

_df_fixed8 = df_fixed8.copy()

# for name, _df in _df_fixed8.groupby(('Dystonia','PID')):
#     _df_fixed8.fillna(method='ffill', inplace=True)

_df_fixed8.fillna(method='ffill', inplace=True)
# print("After filling missing values:")
_df_fixed8.isna().sum()[::2]
pass

In [37]:
# Divide by shoulder length and _subtract the Nose coordinate 
norm_df_fixed8 = _df_fixed8.copy()
for name, _df in norm_df_fixed8.groupby(('Dystonia','PID')):
    
    diff = (_df[('LSHO', 'x')].mean() - _df[('RSHO', 'x')].mean())  # we want this distance to be 10
    xvals = _df.iloc[:, _df.columns.get_level_values(1)=='x']
    xvals = (xvals- _df[('NOSE', 'x')].mean())/diff
    yvals = _df.iloc[:, _df.columns.get_level_values(1)=='y']
    yvals = (yvals- _df[('NOSE', 'y')].mean())/diff
    
    norm_df_fixed8.update(xvals)
    norm_df_fixed8.reset_index()
    norm_df_fixed8.update(yvals)
    norm_df_fixed8.reset_index()

# norm_df_fixed8

# norm_df_fixed8.to_hdf("./norm_df_fixed8.hdf", key='han')

Part   Dystonia            NOSE                NECK                RSHO  \
Coord       PID   FID         x         y         x         y         x   
0          1001  8300 -0.006668 -0.208436  0.045130  0.405509 -0.415353   
9          1001  8309 -0.006489 -0.208805  0.044832  0.405533 -0.415401   
18         1001  8318  0.010347 -0.242592  0.028937  0.405747 -0.415735   
27         1001  8327  0.011230 -0.258800  0.044772  0.405712 -0.415544   
36         1001  8336  0.010371 -0.242690  0.045058  0.405759 -0.415270   
...         ...   ...       ...       ...       ...       ...       ...   
498840        9  9934  0.053820  0.016427  0.091700  0.629553 -0.406341   
498853        9  9947  0.091360  0.016036  0.129678  0.629821 -0.380850   
498867        9  9961  0.168067  0.017110  0.181108  0.693514 -0.316960   
498880        9  9974  0.244238  0.004040  0.256984  0.693425 -0.253142   
498894        9  9988  0.270355  0.004276  0.244730  0.655045 -0.227650   

Part                  LSHO                REYE                LEYE            \
Coord          y         x         y         x         y         x         y   
0       0.490420  0.556755  0.302914 -0.107403 -0.293571  0.146365 -0.276077   
9       0.490611  0.556636  0.302652 -0.107915 -0.277760  0.130578 -0.276497   
18      0.507424  0.556541  0.302628 -0.107903 -0.293473  0.146484 -0.277279   
27      0.507257  0.556636  0.302771 -0.091567 -0.293986  0.147343 -0.277696   
36      0.507269  0.556648  0.302724 -0.107426 -0.293888  0.146651 -0.277378   
...          ...       ...       ...       ...       ...       ...       ...   
498840  0.642621  0.589732  0.616513 -0.049236 -0.085707  0.155491 -0.098545   
498853  0.655259  0.640054  0.616531 -0.036365 -0.085655  0.168469 -0.098353   
498867  0.718774  0.679158  0.642513  0.052971 -0.060113  0.257708 -0.085470   
498880  0.718836  0.755275  0.642371  0.117013 -0.085221  0.334182 -0.097772   
498894  0.719283  0.678720  0.603829  0.142897 -0.097761  0.359165 -0.085671   

Part        REAR                LEAR            
Coord          x         y         x         y  
0      -0.210212 -0.174377  0.352174 -0.071942  
9      -0.227227 -0.157386  0.351864 -0.072276  
18     -0.227490 -0.140669  0.335278 -0.106226  
27     -0.227394 -0.140604  0.368689 -0.105994  
36     -0.210677 -0.140737  0.368928 -0.089482  
...          ...       ...       ...       ...  
498840 -0.176533  0.016854  0.333941 -0.009091  
498853 -0.164109  0.029146  0.346321  0.003575  
498867 -0.074629  0.067798  0.398179  0.016522  
498880 -0.036079  0.080134  0.461819  0.029436  
498894  0.002292  0.042275  0.461756  0.003777  

[46200 rows x 18 columns]

In [ ]:
# see individual data
norm_df_fixed8[norm_df_fixed8[('Dystonia','PID')]==9]

Part   Dystonia            NOSE                NECK                RSHO  \
Coord       PID   FID         x         y         x         y         x   
497306        9  8400 -0.048897 -0.034538 -0.035838  0.617192 -0.534344   
497319        9  8413 -0.138206 -0.008681 -0.010615  0.616522 -0.508683   
497332        9  8426 -0.265861  0.042352  0.039984  0.629098 -0.469972   
497346        9  8440 -0.317282  0.105463  0.040493  0.680340 -0.457449   
497359        9  8453 -0.355939  0.105427  0.040038  0.654822 -0.457807   
...         ...   ...       ...       ...       ...       ...       ...   
498840        9  9934  0.053820  0.016427  0.091700  0.629553 -0.406341   
498853        9  9947  0.091360  0.016036  0.129678  0.629821 -0.380850   
498867        9  9961  0.168067  0.017110  0.181108  0.693514 -0.316960   
498880        9  9974  0.244238  0.004040  0.256984  0.693425 -0.253142   
498894        9  9988  0.270355  0.004276  0.244730  0.655045 -0.227650   

Part                  LSHO                REYE                LEYE            \
Coord          y         x         y         x         y         x         y   
497306  0.642344  0.474904  0.591280 -0.163573 -0.111116  0.052641 -0.136758   
497319  0.629178  0.499904  0.591155 -0.240003 -0.099028 -0.023110 -0.111445   
497332  0.629812  0.538552  0.616807 -0.317299 -0.034565 -0.163698 -0.059717   
497346  0.668014  0.550896  0.655241 -0.354929  0.003698 -0.215128  0.003452   
497359  0.655215  0.538025  0.642263 -0.368238  0.003553 -0.253785  0.003703   
...          ...       ...       ...       ...       ...       ...       ...   
498840  0.642621  0.589732  0.616513 -0.049236 -0.085707  0.155491 -0.098545   
498853  0.655259  0.640054  0.616531 -0.036365 -0.085655  0.168469 -0.098353   
498867  0.718774  0.679158  0.642513  0.052971 -0.060113  0.257708 -0.085470   
498880  0.718836  0.755275  0.642371  0.117013 -0.085221  0.334182 -0.097772   
498894  0.719283  0.678720  0.603829  0.142897 -0.097761  0.359165 -0.085671   

Part        REAR                LEAR            
Coord          x         y         x         y  
497306 -0.291388  0.003582  0.181045 -0.021665  
497319 -0.329893 -0.084844  0.219095 -0.021989  
497332 -0.329893 -0.084844  0.155456  0.016412  
497346 -0.329893 -0.084844  0.141951  0.067457  
497359 -0.329893 -0.084844  0.078900  0.042462  
...          ...       ...       ...       ...  
498840 -0.176533  0.016854  0.333941 -0.009091  
498853 -0.164109  0.029146  0.346321  0.003575  
498867 -0.074629  0.067798  0.398179  0.016522  
498880 -0.036079  0.080134  0.461819  0.029436  
498894  0.002292  0.042275  0.461756  0.003777  

[120 rows x 18 columns]

In [79]:
# plot linePlots_all for all the data
for pid, _df in norm_df_fixed8.groupby(('Dystonia','PID')):
    # pid= 9
    score = GDRS[GDRS['PID']==str(pid)].values[0][4:6]
    ax = plt.subplot(211, )
    ax = norm_df_fixed8[norm_df_fixed8[('Dystonia','PID')]==pid].loc[:,  norm_df_fixed8.columns.get_level_values(1).isin(['x'])].plot(figsize=(20,10),  ylim=(-0.8,0.8), ax=ax)
    plt.ylabel("X-coordinate")

    ax2 = plt.subplot(212, sharex=ax)
    norm_df_fixed8[norm_df_fixed8[('Dystonia','PID')]==pid].loc[:,  norm_df_fixed8.columns.get_level_values(1).isin(['y'])].plot(figsize=(20,10),  ylim=(-0.8,0.8), ax=ax2)
    ax2.text(0, -0.1, 'PID: '+ str(pid)+' Larynx: '+ str(score[0])+ ' Neck: '+ str(score[1]),  transform=ax.transAxes, fontsize=20)
    plt.ylabel("Y-coordinate")
    plt.xlabel("Frames -->")

    # plt.show()
    plt.savefig('D:\DystoniaCoalition\processed\linePlots_all\PID_'+ str(pid)+'_Larynx'+ str(score[0])+ '_Neck'+ str(score[1]) +'.png', bbox_inches='tight')
    plt.clf()


<Figure size 1440x720 with 0 Axes>

In [38]:
GDRS[GDRS['PID'].isin(list(norm_df_fixed8[('Dystonia', 'PID')].unique().astype(str)))].iloc[:,4:6]